In [1]:
import cv2
import os
import json
import subprocess

In [ ]:
input_dir = r"C:\Users\Andrew Jeon\OneDrive\Desktop\ICTAM\videos"
img_dir = r"C:\Users\Andrew Jeon\OneDrive\Desktop\ICTAM\data\images"

In [3]:
### GENERATE IMAGES ###
# downloaded ArtosisCasts up to Dark Rizzler
# yt-dlp -f "bv[height=1080][ext=mp4]" --remux-video mp4 -o "%(title)s.%(ext)s" -a links.txt
# yt-dlp -f "bv[height=1080][ext=mp4]" --remux-video mp4 -o "%(title)s.%(ext)s" "https://www.youtube.com/watch?v=WNr9fuccoxg&t=169s"
# GENERATE MINIMAP FRAMES WITH CROP FOR 1080P video

for filename in os.listdir(input_dir):
    if filename.lower().endswith('.mp4'):
        video_path = os.path.join(input_dir, filename)
        name, ext = os.path.splitext(filename)
        img_name = os.path.join(img_dir, f"{name}_%03d.jpg")

        cmd = [
            r"C:\Users\Andrew Jeon\OneDrive\Desktop\ffmpeg-7.0.2-essentials_build\bin\ffmpeg.exe",
            "-i", video_path,
            "-vf", "fps=1/60,crop=295:295:10:775",  #775-1070 height, 10-305 width
            "-q:v", "2",  # high quality
            img_name
        ]
        subprocess.run(cmd, check=True)

In [ ]:
### GENERATE ANNOTATIONS ###

# VQA Train on image + Questions + Answers
# BLIP-2 + Flan-T5 Base Trained   vs   BLIP-2 + Flan-T5 XL Baseline
# Large and XL are too big to train
#
# _________________________
# DATA
# _________________________  
# art1-art16 just 16 videos ~ 300 clips (1min)

# project/
# ├── images/
# │   ├── art1_001.jpg
# │   ├── art1_002.jpg
# ├── your_annotations.json

# Clips may make more sense for strategic reasoning, temporal aspect. Later on sample 8 frames per clip with one caption per clip to get temporal reasoning.
# data/
# ├── images/
# │   ├── art1_c01/
# │   │   ├── frame_00.jpg  ← earliest (e.g., t=0s)
# │   │   ├── frame_01.jpg
# │   │   ├── ...
# │   │   └── frame_07.jpg  ← latest (e.g., t=60s)
# │   ├── art1_c02/
# │   │   ├── frame_00.jpg
# │   │   └── ...
# │   └── ...
# ├── minimap_annot.json
# │   {
# │     "art1_c01": "Red expands while Blue gathers forces near the center.",
# │     "art1_c02": "Blue scouts the bottom corner as Red fortifies their second base.",
# │   }

output_dir = r"C:\Users\Andrew Jeon\OneDrive\Desktop\ICTAM\data\minimap_annot.json"

entries = []
for filename in sorted(os.listdir(img_dir)):
    if filename.lower().endswith(".jpg"):
        entries.append({
            "image": filename,
            "text_input": "Who is winning? Describe the minimap.",
            "text_output": "",
        })

with open(output_dir, "w") as f:
    json.dump(entries, f, indent=4)

print(f"Saved {len(entries)} prompt-response entries to {output_dir}")

Saved 1511 prompt-response entries to C:\Users\Andrew Jeon\OneDrive\Desktop\AISAA\data\minimap_annot.json
